In [ ]:
!pip install azure-ai-formrecognizer==3.2.0b3

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient, DocumentModelAdministrationClient
import os
import pandas as pd

In [ ]:
os.environ["AZURE_FORM_RECOGNIZER_KEY"] = "Form Recognizer Key"
os.environ["AZURE_FORM_RECOGNIZER_ENDPOINT"] = "https://{endpoint}.cognitiveservices.azure.com/"

In [ ]:
endpoint = os.environ["AZURE_FORM_RECOGNIZER_ENDPOINT"]
key = os.environ["AZURE_FORM_RECOGNIZER_KEY"]

document_admin_client = DocumentModelAdministrationClient(endpoint=endpoint, credential=AzureKeyCredential(key))
models = document_admin_client.list_models()
for model in models:
    print("{} | {}".format(model.model_id, model.description))




In [ ]:
document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key))

path_to_sample_documents = os.path.abspath(
        os.path.join(
            os.getcwd(),
            "Demo Data/Car Maintenance/Test/Ignite.png",
        )
    )

with open(path_to_sample_documents, "rb") as f:
        poller = document_analysis_client.begin_analyze_document(
            "CarMaint", document=f)
result = poller.result()
for idx, document in enumerate(result.documents):
        print("--------Analyzing document #{}--------".format(idx + 1))
        print("Document has type {}".format(document.doc_type))
        print("Document has confidence {}".format(document.confidence))
        print("Document was analyzed by model with ID {}".format(result.model_id))
        for name, field in document.fields.items():
            field_value = field.value if field.value else field.content
            
            
            if field.value_type == "list":
                df_list  = []
                for row in field.value:
                    a_row = {}
                    for key, value in row.value.items():
                        a_row[key] = value.content
                    df_list.append(a_row)
                
                df = pd.DataFrame(df_list)
                display(df)
            else:
                print("......found field of type '{}' with value '{}' and with confidence {}".format(field.value_type, field_value, field.confidence))
            